# Phân loại bình luận phim là tích cực hay tiêu cực

## Import các thư viện cần thiết

- Tải thư viện (dùng lệnh !pip install tên_thư_viện)
- dấu ! ở đầu dòng là lệnh của jupyter notebook -> chạy lệnh này trên terminal

In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn

  Preparing metadata (setup.py) ... - done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2360 sha256=38795db8aba9a3834f41d1f11d2ec5b9c0e1cc584ed7a618f8070e572836b3b9
  Stored in directory: /root/.cache/pip/wheels/b2/af/1b/ac28f3fb36a8428e3089acdd913e9ee1808e781e3ff6ce2929
Successfully built sklearn


In [2]:
!dir
# lệnh dir để xem các file trong thư mục hiện tại

__notebook__.ipynb


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mydatafiles123/IMDB Dataset.csv


## Đọc dữ liệu từ file csv và tiền xử lý

In [4]:
dataset = pd.read_csv(os.path.join(dirname, "IMDB Dataset.csv")).head(4000)
# Cột review là nội dung review
# Cột sentiment là nhãn của review (positive: tích cực, negative: tiêu cực, neutral: trung lập)

# In ra 5 dòng đầu tiên của dataset
dataset.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
reviews = np.array(dataset['review'])  # lấy ra cột review -> chuyển về dạng numpy array
sentiments = np.array(dataset['sentiment'])  # lấy ra cột sentiment -> chuyển về dạng numpy array


# Xử lý dữ liệu (chuyển về chữ thường)
def text_preprocessing(text):
    # Xử lý dữ liệu:
    # 1. Chuyển về chữ thường
    text.lower()

    # 2. Xóa các ký tự đặc biệt (Nếu cần) Xoá HTML tags, ...

    return text


X = []  # tập dữ liệu (các review)
y = []  # tập nhãn

for i in range(len(reviews)):
    X.append(text_preprocessing(reviews[i]))  # thêm review đã xử lý vào tập dữ liệu
    y.append(sentiments[i])  # thêm nhãn vào tập nhãn


In [6]:
print("Tập dữ liệu X[0]: ", X[0])
print("Tập nhãn y[0]: ", y[0])

Tập dữ liệu X[0]:  One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the sho

## Tạo các đặc trưng (feature) cho mô hình học máy

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo model TF-IDF
tfidf_model = TfidfVectorizer(
    stop_words='english',  # loại bỏ các stop words
    use_idf=True,  # sử dụng TF-IDF
    min_df=0.0,  # tần suất xuất hiện tối thiểu của từ trong tập dữ liệu
    max_df=1.0,  # tần suất xuất hiện tối đa của từ trong tập dữ liệu
    ngram_range=(1, 2),  # sử dụng n-gram
    sublinear_tf=True  # sử dụng sublinear_tf
)
tfidf_feature_vector = tfidf_model.fit_transform(
    X  # tập dữ liệu train,
)  # fit model với tập train
print("Số lượng từ trong tập dữ liệu: ", len(tfidf_model.vocabulary_))

Số lượng từ trong tập dữ liệu:  370410


In [8]:
# Tập nhãn phải là số nguyên (0, 1, 2, ...) nên cần chuyển về dạng số nguyên để huấn luyện mô hình học máy
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_lable_encoding = le.fit_transform(y)

In [9]:
# Chia tập dữ liệu thành tập train và tập test
from sklearn.model_selection import train_test_split

train_reviews, test_reviews, train_sentiments, test_sentiments = train_test_split(
    tfidf_feature_vector.toarray(), # tập dữ liệu
    y_lable_encoding, # tập nhãn
    test_size=0.2, # tỉ lệ dữ liệu test
    random_state=21 # seed
)

## Model Training, Prediction and Performance Evaluation

In [10]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB() # Tạo model Naive Bayes
nb.fit(train_reviews, train_sentiments) # Huấn luyện mô hình với tập train

GaussianNB()

In [11]:
from sklearn.metrics import classification_report
# Dự đoán nhãn của từ tập test
test_sentiments_pred = nb.predict(test_reviews)

# Đánh giá mô hình so sánh với dự đoán và nhãn đúng
print(classification_report(test_sentiments, test_sentiments_pred))

              precision    recall  f1-score   support

           0       0.68      0.71      0.70       398
           1       0.70      0.66      0.68       402

    accuracy                           0.69       800
   macro avg       0.69      0.69      0.69       800
weighted avg       0.69      0.69      0.69       800



In [12]:
input = "I hate this movie" # Nhập vào đoạn review mà muốn phân loại là bình luận tích cực hay tiêu cực

input = [text_preprocessing(input)] # Xử lý test, đưa vào list 
# Chuyển về dạng vector
input_tfidf_feature_vector = tfidf_model.transform(input).toarray()

# Dự đoán nhãn
input_sentiments_pred = nb.predict(input_tfidf_feature_vector)

if input_sentiments_pred == 1:
    print("Review tích cực") # Dự đoán nhãn là tích cực 
else: 
    print("Bình luận tiêu cực") # Dự đoán nhãn là tiêu cực

Bình luận tiêu cực
